# Media Pipe explained

In [14]:
import os
import numpy as np
import matplotlib.pyplot as plt

import cv2
import mediapipe as mp


In [15]:
def print_elements(x, in_row = 5):
    names = [e.name for e in x]
    names_length = len(names)
    for i in range(names_length // in_row):
        print(names[(i * in_row):min((i + 1) * in_row, names_length)])

Here we are preparing media pipe tools
for furhter usage

In [16]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

# Test

Point names to be detected

In [17]:
print('Hands parts')
print(type(mp_hands.HandLandmark))
print_elements(mp_hands.HandLandmark)

print('\nFace parts')
print(type(mp_face.FaceMesh))
print(mp_face.FaceMesh)

print('\nPose parts')
print(type(mp_pose.PoseLandmark))
print_elements(mp_pose.PoseLandmark)

Hands parts
<class 'enum.EnumMeta'>
['WRIST', 'THUMB_CMC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP']
['INDEX_FINGER_MCP', 'INDEX_FINGER_PIP', 'INDEX_FINGER_DIP', 'INDEX_FINGER_TIP', 'MIDDLE_FINGER_MCP']
['MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_DIP', 'MIDDLE_FINGER_TIP', 'RING_FINGER_MCP', 'RING_FINGER_PIP']
['RING_FINGER_DIP', 'RING_FINGER_TIP', 'PINKY_MCP', 'PINKY_PIP', 'PINKY_DIP']

Face parts
<class 'type'>
<class 'mediapipe.python.solutions.face_mesh.FaceMesh'>

Pose parts
<class 'enum.EnumMeta'>
['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER']
['RIGHT_EYE', 'RIGHT_EYE_OUTER', 'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT']
['MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW']
['LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX']
['RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP']
['LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL']
